# Connexion à l'API France Travail

URL : 

https://francetravail.io/data/api/offres-emploi

https://francetravail.io/data/api/offres-emploi/documentation#/

## Imports

In [36]:
import http.client
import requests
import json
import pandas as pd
import os

In [63]:
# ---------------------------
# CONFIGURATION
# ---------------------------
# France Travail (ex-Pôle emploi)
# FT_CLIENT_ID = Defined in .env
# FT_CLIENT_SECRET = Defined in .env
# FT_SCOPE = Defined in .env

##################  VARIABLES  ##################
FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
FT_SCOPE = os.environ.get("FT_SCOPE")

# Paramètres de recherche
JOB_QUERY = "data engineer"
COMMUNE = "78300"
DISTANCE = 500
BLOC_PAGINATION = 50

MAX_PAGES = 20   # Limiter le nombre de pages récupérées

# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

# ---------------------------
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_jobs(token, max_pages=MAX_PAGES):
    headers = {"Authorization": f"Bearer {token}"}
    all_jobs = []
    b_stop_criteria = False
    
    for page in range(1, max_pages + 1):
        if b_stop_criteria == False:    
            url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
            params = {
                "motsCles": JOB_QUERY,
                "commune": COMMUNE,
                "distance" : DISTANCE,
                "range": f"{(page-1)*BLOC_PAGINATION}-{page*BLOC_PAGINATION-1}"  # pagination par blocs de 50
            }
            r = requests.get(url, headers=headers, params=params)
            r.raise_for_status()
            data = r.json()
            offres = data.get("resultats", [])
            for o in offres:
                all_jobs.append({
                    "source": "France Travail",
                    "titre": o.get("intitule"),
                    "entreprise": o.get("entreprise", {}).get("nom"),
                    "lieu": o.get("lieuTravail", {}).get("libelle"),
                    "date": o.get("dateCreation"),
                    "url": o.get("origineOffre", {}).get("urlOrigine"),
                })
            print(len(offres))
            
            if len(offres) < BLOC_PAGINATION:
                b_stop_criteria = True
            
    return all_jobs

# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    print("Authentification France Travail...")
    token = get_ft_token()

    print("Récupération des offres France Travail...")
    ft_jobs = fetch_france_travail_jobs(token)

    print("Affichage Extract offres France Travail...")
    df = pd.DataFrame(ft_jobs)
    display(df.head(10))

    # Export vers CSV
    df.to_csv("../data/raw_data/offres_emploi.csv", index=False, encoding="utf-8")
    print(f"{len(ft_jobs)} offres uniques exportées dans offres_emploi.csv ✅")


Authentification France Travail...
Récupération des offres France Travail...
50
50
50
50
50
50
50
50
16
Affichage Extract offres France Travail...


,source,titre,entreprise,lieu,date,url
0,France Travail,Data Engineer - H/F - CDI (H/F),POLYCEA,75 - PARIS 09,2025-09-12T15:46:22.604Z,https://candidat.francetravail.fr/offres/reche...
1,France Travail,Data engineer,E-KENT,75 - Paris 13e Arrondissement,2025-09-12T16:20:49.815Z,https://candidat.francetravail.fr/offres/reche...
2,France Travail,Data engineer,EPSYL,33 - Bordeaux,2025-09-12T16:01:21.737Z,https://candidat.francetravail.fr/offres/reche...
3,France Travail,Data Engineer Kafka / BigQuery (H/F),SEGMENT ELITE,75 - PARIS 09,2025-09-11T18:06:20.094Z,https://candidat.francetravail.fr/offres/reche...
4,France Travail,Data Engineer - EMEA (H/F),DOLEAD,75 - PARIS 02,2025-09-11T09:48:37.782Z,https://candidat.francetravail.fr/offres/reche...
5,France Travail,Data Engineer SQL (H/F),SEGMENT ELITE,75 - PARIS 09,2025-09-09T13:23:39.343Z,https://candidat.francetravail.fr/offres/reche...
6,France Travail,Data Engineer (H/F),SEGMENT ELITE,75 - PARIS 09,2025-09-09T13:13:35.180Z,https://candidat.francetravail.fr/offres/reche...
7,France Travail,Data engineer (H/F),AMILTONE,59 - VILLENEUVE D ASCQ,2025-09-09T10:21:15.382Z,https://candidat.francetravail.fr/offres/reche...
8,France Travail,Software Engineer Data & AI (H/F),HIPAY,44 - Nantes,2025-09-09T11:28:00.390Z,https://candidat.francetravail.fr/offres/reche...
9,France Travail,Data engineer (H/F),None,69 - LYON 07,2025-09-08T11:38:56.778Z,https://candidat.francetravail.fr/offres/reche...


416 offres uniques exportées dans offres_emploi.csv ✅
